In [3]:
# TU LIMPIEZA Y ANÁLISIS AQUÍ
import pandas as pd
import numpy as np
import seaborn as sns
import cleaning

In [2]:
tvs_df=pd.read_csv('datos/tvs.csv')
tvs_df

,_id,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,...,original_language,original_name,overview,popularity,poster_path,production_companies,status,tagline,vote_average,vote_count
0,6457d26c7b901e08b141869d,[],1996-10-09,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",13,True,2022-02-16,How do you like Wednesday?,568.0,4,...,ja,水曜どうでしょう,How do you like Wednesday? was a Japanese tele...,21.207,/i2MwS6U0XzD8ad6aS3HiTNKz8ov.jpg,[],Returning Series,NaN,9.200,2
1,6457d26c7b901e08b141867a,"[{""id"":19303,""name"":""Kevin Smith"",""_id"":{""$oid...",2000-05-31,"[{""id"":16,""name"":""Animation"",""_id"":{""$oid"":""64...",2,False,2002-12-22,Clerks,6.0,1,...,en,Clerks,The continuing adventures of store clerks Dant...,10.620,/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,"[{""id"":1558,""name"":""Touchstone Television"",""lo...",Canceled,NaN,6.897,78
2,6457d26c7b901e08b141869f,[],2006-08-20,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",14,False,2006-11-11,Shuriken School,26.0,1,...,en,Shuriken School,Shuriken School is a British animated series t...,3.034,/khXHvYKAvGqjtnKoLVYZr8r5yKD.jpg,"[{""id"":75257,""name"":""Zinkia Entertainment"",""lo...",Ended,NaN,5.500,6
3,6457d26c7b901e08b14186a5,"[{""id"":1074129,""name"":""Fred Rogers"",""_id"":{""$o...",1968-02-19,"[{""id"":10762,""name"":""Kids"",""_id"":{""$oid"":""6457...",15,False,2001-09-07,Mister Rogers' Neighborhood,850.0,31,...,en,Mister Rogers' Neighborhood,Mister Rogers' Neighborhood is an American chi...,19.167,/4Gz46uJg6MFaM4xeblbVNSxCyTr.jpg,"[{""id"":16021,""name"":""WQED"",""logo_path"":""/2bjAi...",Ended,Won't you be my neighbor?,4.977,88
4,6457d26c7b901e08b14186ab,"[{""id"":1211910,""name"":""Richard R. Lee"",""_id"":{...",NaN,[],16,False,2005-09-09,Sidewalks: Video Nite,399.0,11,...,en,Sidewalks: Video Nite,Sidewalks: Video Nite is an American televisio...,2.910,NaN,[],Ended,NaN,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150797,645c00b1378e4a37f5092e21,[],2015-06-12,"[{""id"":99,""name"":""Documentary"",""_id"":{""$oid"":""...",226231,False,2015-06-13,北京之夜,4.0,1,...,zh,北京之夜,NaN,0.000,NaN,[],Ended,NaN,0.000,0
150798,645c00b1378e4a37f5092e23,[],2014-07-29,"[{""id"":99,""name"":""Documentary"",""_id"":{""$oid"":""...",226232,False,2014-09-02,北洋海军兴亡史,6.0,1,...,zh,北洋海军兴亡史,NaN,0.000,/rcNz0Amku0XWOC3LuLfRHiKYqS3.jpg,[],Ended,NaN,0.000,0
150799,645c00b1378e4a37f5092e25,[],2023-05-09,[],226233,True,2023-05-09,Turkey: Empire of Erdogan,1.0,1,...,en,Turkey: Empire of Erdogan,A few months after his brutal crackdown on pro...,0.000,/l7rcVP8Q5yY4M7O6rJSstV7eIin.jpg,[],Returning Series,NaN,0.000,0
150800,645c00b1378e4a37f5092e26,[],2023-01-24,[],226234,True,2023-01-24,Woooooo! Becoming Ric Flair,1.0,1,...,en,Woooooo! Becoming Ric Flair,Ric Flair Biography,0.000,NaN,[],Returning Series,NaN,0.000,0


In [5]:
tvs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150802 entries, 0 to 150801
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   150802 non-null  object 
 1   created_by            150802 non-null  object 
 2   first_air_date        119932 non-null  object 
 3   genres                150802 non-null  object 
 4   id                    150802 non-null  int64  
 5   in_production         150802 non-null  bool   
 6   last_air_date         122108 non-null  object 
 7   name                  150796 non-null  object 
 8   number_of_episodes    150168 non-null  float64
 9   number_of_seasons     150802 non-null  int64  
 10  origin_country        150802 non-null  object 
 11  original_language     150802 non-null  object 
 12  original_name         150796 non-null  object 
 13  overview              84623 non-null   object 
 14  popularity            150802 non-null  float64
 15  

In [17]:
muestra_tvs_df = tvs_df.sample(20)
muestra_tvs_df

,_id,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,...,original_language,original_name,overview,popularity,poster_path,production_companies,status,tagline,vote_average,vote_count
139999,64585b2c7b901e08b14757ce,"[{""id"":1702940,""name"":""Anne Landois"",""_id"":{""$...",NaN,"[{""id"":10759,""name"":""Action & Adventure"",""_id""...",213845,True,NaN,66-5,1.0,1,...,fr,66-5,NaN,0.600,NaN,"[{""id"":185485,""name"":""Sortilèges Productions"",...",In Production,NaN,0.000,0
113242,645827de7b901e08b1464bca,[],2014-11-22,[],132201,True,2016-12-11,Anna und die wilden Lieder,22.0,1,...,de,Anna und die wilden Lieder,NaN,0.600,NaN,[],Returning Series,NaN,0.000,0
128152,645850fb7b901e08b146dfc5,[],2022-04-04,[],197874,True,2022-09-24,Mithai,149.0,1,...,hi,मिठाई,"Mithai, a gifted sweet maker, is on a quest to...",0.600,NaN,[],Returning Series,NaN,2.000,1
98068,64581c007b901e08b145a659,[],2016-12-14,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""64581...",113792,False,2016-12-14,"Not Today, Bianca",2.0,1,...,en,"Not Today, Bianca",Drag queen Bianca Del Rio has arrived in Los A...,1.258,/m2UmlHliMcJYdQRkiAntHnjtS3L.jpg,"[{""id"":1421,""name"":""World of Wonder"",""logo_pat...",Ended,NaN,1.000,1
7588,6457d5ae7b901e08b141ecc2,[],NaN,[],8061,False,NaN,Ποιος θέλει να γίνει εκατομμυριούχος,0.0,0,...,el,Ποιος θέλει να γίνει εκατομμυριούχος,Based on the British Who Wants to be a Million...,0.600,NaN,[],Ended,NaN,0.000,0
10337,6457d7857b901e08b1420845,[],2001-08-21,"[{""id"":18,""name"":""Drama"",""_id"":{""$oid"":""6457d7...",10949,False,2001-09-07,Beech is Back,6.0,1,...,en,Beech is Back,Beech is Back was a storyline in the long-runn...,1.205,NaN,[],Ended,NaN,0.000,0
138428,645859ff7b901e08b14749ad,[],NaN,[],212096,True,2022-08-13,Kid Temper Tantrum,1.0,1,...,en,Kid Temper Tantrum,"Get to know the family, of stressed out dad, M...",0.600,NaN,[],Returning Series,NaN,0.000,0
61703,6457fd607b901e08b143e746,"[{""id"":964786,""name"":""Laurent Sarfati"",""_id"":{...",2016-11-22,"[{""id"":16,""name"":""Animation"",""_id"":{""$oid"":""64...",68888,False,2022-11-09,Lastman,32.0,2,...,fr,Lastman,"Paxtown, a corrupt capital, torn by crime and ...",5.760,/8ZGT8RdjNOczejJMfqFw97BdPOR.jpg,"[{""id"":33963,""name"":""Everybody on Deck"",""logo_...",Ended,NaN,7.643,21
143941,64585e487b901e08b147806b,"[{""id"":1922681,""name"":""Clémence Madeleine-Perd...",NaN,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""64585...",218575,True,NaN,Irrésistible,6.0,1,...,fr,Irrésistible,NaN,0.600,NaN,"[{""id"":191300,""name"":""Jour Premier Production""...",Returning Series,NaN,0.000,0
18395,6457dd007b901e08b14254b2,[],NaN,[],19520,False,NaN,GSN Video Games,0.0,0,...,en,GSN Video Games,GSN Video Games was a short-lived block of sho...,0.600,NaN,[],Ended,NaN,0.000,0


In [6]:
import re

# Función para extraer el nombre de la compani
def extraer_nombres(df):
    import re
    lista_names = df[columns]
    lista_nombres = []
    opciones = df[columns]
    columna = input(f'escribe la columna que vaya aplicar la funcion: /n {opciones}')
    nueva_columna = input('escribre nombre de la nueva columna') 
    for name in lista_names:
        match = re.search(r'"name":"([^"]*)"', name)
        if match:
            lista_name = match.group(1)
            lista_nombres.append(lista_name)
    return lista_nombres
# Crear la nueva columna
    df['companias_nombre'] = df.apply(extract_company_name, axis=1)

In [7]:
tvs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150802 entries, 0 to 150801
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   150802 non-null  object 
 1   created_by            150802 non-null  object 
 2   first_air_date        119932 non-null  object 
 3   genres                150802 non-null  object 
 4   id                    150802 non-null  int64  
 5   in_production         150802 non-null  bool   
 6   last_air_date         122108 non-null  object 
 7   name                  150796 non-null  object 
 8   number_of_episodes    150168 non-null  float64
 9   number_of_seasons     150802 non-null  int64  
 10  origin_country        150802 non-null  object 
 11  original_language     150802 non-null  object 
 12  original_name         150796 non-null  object 
 13  overview              84623 non-null   object 
 14  popularity            150802 non-null  float64
 15  

In [8]:
tvs_df.drop(columns=['poster_path','_id','tagline'],axis=1 ,inplace=True) # columna imagem no aporta nada, la dropeo
tvs_df.sample(3)

,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,origin_country,original_language,original_name,overview,popularity,production_companies,status,vote_average,vote_count
19570,[],NaN,[],20898,True,NaN,Wimps to Warriors,1.0,1,"[""GB""]",en,Wimps to Warriors,NaN,0.600,[],Returning Series,0.0,0
135359,[],2022-09-22,[],208459,True,2022-10-13,さよならの向う側,6.0,1,"[""JP""]",ja,さよならの向う側,NaN,0.600,"[{""id"":11727,""name"":""Yomiuri Telecasting Corpo...",Returning Series,0.0,0
99013,[],2020-12-01,"[{""id"":18,""name"":""Drama"",""_id"":{""$oid"":""64581c...",114972,False,2020-12-01,AWEL EL SATR,1.0,1,"[""EG""]",ar,أول السطر,a series about love and death,0.971,"[{""id"":127387,""name"":""Synergy"",""logo_path"":""/1...",Ended,2.0,1


In [10]:
tvs_df.production_companies.sample(3)

109135                                                   []
6538                                                     []
93171     [{"id":79261,"name":"TVB","logo_path":"/4OKzka...
Name: production_companies, dtype: object

In [11]:
tvs_df['generos'] = tvs_df['genres'].apply(cleaning.extraer_nombres)
tvs_df['generos'].sample(3)


99057          Drama
87775    Desconocido
56705          Crime
Name: generos, dtype: object

In [12]:
tvs_df['creado_por'] = tvs_df['created_by'].apply(cleaning.extraer_nombres)
tvs_df.sample(3)

,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,origin_country,original_language,original_name,overview,popularity,production_companies,status,vote_average,vote_count,generos,creado_por
14384,[],1973-02-05,[],15122,False,1975-10-24,The Wombles,125.0,5,"[""GB""]",en,The Wombles,The Wombles is a stop motion animated British ...,7.517,"[{""id"":74594,""name"":""FilmFair"",""logo_path"":nul...",Ended,7.200,9,Desconocido,Desconocido
67770,"[{""id"":1676246,""name"":""Park Hae-young"",""_id"":{...",2018-03-21,"[{""id"":18,""name"":""Drama"",""_id"":{""$oid"":""645802...",76662,False,2018-05-17,My Mister,16.0,1,"[""KR""]",ko,나의 아저씨,"In a world that is less than kind, a young wom...",20.048,"[{""id"":70599,""name"":""Chorokbaem Media"",""logo_p...",Ended,8.299,92,Drama,"Park Hae-young, Kim Won-suk"
1842,[],1991-01-17,[],1971,False,NaN,Sunrise,0.0,0,"[""AU""]",en,Sunrise,Sunrise is an Australian breakfast television ...,0.600,[],Ended,0.000,0,Desconocido,Desconocido


In [13]:
tvs_df['productoras'] = tvs_df['production_companies'].apply(cleaning.extraer_nombres)
tvs_df.sample(3)

,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,origin_country,...,original_name,overview,popularity,production_companies,status,vote_average,vote_count,generos,creado_por,productoras
96929,[],2003-04-15,[],112374,True,2003-04-15,Pittanko Kan Kan,1.0,1,"[""JP""]",...,ぴったんこカン・カン,NaN,1.01,[],Returning Series,0.0,0,Desconocido,Desconocido,Desconocido
129472,[],2022-05-08,[],201575,True,2022-11-27,Sondos,30.0,1,"[""SA""]",...,سندس,A well-known businessman is forced to take car...,0.84,[],Returning Series,7.8,5,Desconocido,Desconocido,Desconocido
106582,[],2013-06-15,"[{""id"":18,""name"":""Drama"",""_id"":{""$oid"":""645822...",124025,True,2013-07-12,คุณชายรณพีร์ สุภาพบุรุษจุฑาเทพ,12.0,1,[],...,คุณชายรณพีร์ สุภาพบุรุษจุฑาเทพ,NaN,0.60,"[{""id"":153133,""name"":""Metta & Mahaniyom Co., L...",Returning Series,0.0,0,Drama,Desconocido,"Metta & Mahaniyom Co., LTD."


In [14]:
tvs_df.isnull().sum()

created_by                  0
first_air_date          30870
genres                      0
id                          0
in_production               0
last_air_date           28694
name                        6
number_of_episodes        634
number_of_seasons           0
origin_country              0
original_language           0
original_name               6
overview                66179
popularity                  0
production_companies        0
status                      0
vote_average                0
vote_count                  0
generos                     0
creado_por                  0
productoras                 0
dtype: int64

In [15]:
tvs_df.drop(columns=['production_companies','created_by','genres'],axis=1 ,inplace=True) # columna imagem no aporta nada, la dropeo
tvs_df.sample(3)

,first_air_date,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,origin_country,original_language,original_name,overview,popularity,status,vote_average,vote_count,generos,creado_por,productoras
12105,NaN,12757,True,2018-01-20,The Passionate Eye,14.0,6,"[""CA""]",en,The Passionate Eye,The Passionate Eye is a Canadian documentary t...,1.400,Returning Series,0.0,0,Documentary,Desconocido,Desconocido
97203,2020-08-25,112797,True,2020-10-06,Crisis in de Tent,7.0,1,[],nl,Crisis in de Tent,NaN,0.600,Returning Series,0.0,0,Desconocido,Desconocido,Desconocido
74643,1993-03-15,85398,False,2003-09-15,Rotlicht,4.0,1,"[""DE""]",de,Rotlicht,NaN,0.927,Ended,0.0,0,Crime,Desconocido,Desconocido


In [24]:
tvs_df.isnull().sum()

first_air_date        30870
id                        0
in_production             0
last_air_date         28694
name                      6
number_of_episodes      634
number_of_seasons         0
origin_country            0
original_language         0
original_name             6
overview              66179
popularity                0
status                    0
vote_average              0
vote_count                0
generos                   0
creado_por                0
productoras               0
dtype: int64

In [25]:
tvs_df['generos'].sample(3)

129938       Desconocido
123048     Drama, Family
14642     Comedy, Family
Name: generos, dtype: object

In [26]:
tvs_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,150802.0,95452.023077,65908.935205,1.0,40398.25,86226.500,132047.75,226235.000
number_of_episodes,150168.0,24.704231,136.447485,0.0,1.00,6.000,20.00,19925.000
number_of_seasons,150802.0,1.550377,3.031528,0.0,1.00,1.000,1.00,345.000
popularity,150802.0,3.487269,22.754721,0.0,0.60,0.737,1.91,2162.502
vote_average,150802.0,2.400667,3.477006,0.0,0.00,0.000,6.00,10.000
vote_count,150802.0,13.953993,193.784426,0.0,0.00,0.000,1.00,21048.000


In [30]:
#tvs_df[tvs_df['name'] != np.()]
tvs_df = tvs_df[~tvs_df['name'].isna()]
#tvs_df[~tvs_df['name'].notna()]


In [31]:
tvs_df.isnull().sum()

first_air_date        30867
id                        0
in_production             0
last_air_date         28693
name                      0
number_of_episodes      634
number_of_seasons         0
origin_country            0
original_language         0
original_name             0
overview              66173
popularity                0
status                    0
vote_average              0
vote_count                0
generos                   0
creado_por                0
productoras               0
dtype: int64

In [33]:
tvs_df.origin_country.sample(3)

95527        []
27358    ["GB"]
5917     ["US"]
Name: origin_country, dtype: object

In [50]:
tvs_df.origin_country = tvs_df.origin_country.replace({"[]": 'Desconocido'})

/tmp/ipykernel_60/2938708518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvs_df.origin_country = tvs_df.origin_country.replace({"[]": 'Desconocido'})


In [53]:
tvs_df.origin_country.sample(3)

16964    Desconocido
22295         ["JP"]
67729         ["US"]
Name: origin_country, dtype: object

In [59]:
tvs_df.origin_country.value_counts()

Desconocido              32649
["US"]                   25166
["GB"]                   12381
["JP"]                   12148
["CN"]                    8318
                         ...  
["DE","US","FR","IN"]        1
["FR","AT"]                  1
["PT","GB","ES"]             1
["ES","DE"]                  1
["CD"]                       1
Name: origin_country, Length: 770, dtype: int64

In [ ]:
tvs_df.to_csv('clean_tvs.csv')

### Preguntas

In [100]:
#¿Cual es la serie con la duración más corta?
#tvs_df.sort_values(by="number_of_episodes",ascending=True).head()# no real
#tengo varios con valor 0
tvs_df.loc[tvs_df['number_of_episodes'] == 0, 'number_of_episodes'] = np.nan
tvs_df.loc[(tvs_df["number_of_episodes"]==tvs_df['number_of_episodes'].min()),["name", "number_of_episodes"]]




,name,number_of_episodes
64,NBA on TNT,1.0
203,Bonds on Bonds,1.0
238,The New Inventors,1.0
263,Korgoth of Barbaria,1.0
286,Fearless,1.0
...,...,...
150785,Shaka iLembe,1.0
150793,超迷你战士,1.0
150795,ポルノグラフィティ「18th ライヴサーキット“暁”」DOCUMENTARY,1.0
150799,Turkey: Empire of Erdogan,1.0


In [101]:
#¿Cual es la serie con la duración más larga?
tvs_df.loc[(tvs_df["number_of_episodes"]==tvs_df['number_of_episodes'].max()),["name", "number_of_episodes"]]


,name,number_of_episodes
82482,Tagesschau,19925.0


In [102]:
#¿Cual es la serie con más temporadas? ¿Y la que tiene más capítulos?
tvs_df.loc[(tvs_df["number_of_episodes"]==tvs_df['number_of_episodes'].max()),["name", "number_of_episodes"]]

,name,number_of_episodes
82482,Tagesschau,19925.0


In [139]:
#¿Cual es el género más estrenado para cada mes y año?
#tvs_df['fecha_inicio'] =  pd.to_datetime(tvs_df['first_air_date'], format="%Y/%m/%d")
#tvs_df['last_air_date']  = tvs_df['last_air_date']..split(' ').apply(lambda x: x[0])
#tvs_df['fecha_fin'] =  pd.to_datetime(tvs_df['last_air_date'], format="%Y/%m/%d")
tvs_df['fecha_fin'] = tvs_df['last_air_date'].str.split('').apply(lambda x: x[0] )#if len(x) > 1 else x)
tvs_df['fecha_fin'].sample(3)


TypeError: 'float' object is not subscriptable